In [1]:
T = [[1,    3, 4   ],[   2, 3,    5],[1, 2, 3,    5],[   2,       5]]

In [2]:
from orangecontrib.associate.fpgrowth import *

In [3]:
itemsets = frequent_itemsets(T, 2)

In [4]:
list(itemsets)

[(frozenset({1}), 2),
 (frozenset({2}), 3),
 (frozenset({3}), 3),
 (frozenset({1, 3}), 2),
 (frozenset({2, 3}), 2),
 (frozenset({5}), 3),
 (frozenset({2, 5}), 3),
 (frozenset({3, 5}), 2),
 (frozenset({2, 3, 5}), 2)]

In [5]:
import Orange

In [6]:
data = Orange.data.Table('zoo')

In [7]:
data

[[1, 0, 0, 1, 0, ... | mammal] {aardvark},
 [1, 0, 0, 1, 0, ... | mammal] {antelope},
 [0, 0, 1, 0, 0, ... | fish] {bass},
 [1, 0, 0, 1, 0, ... | mammal] {bear},
 [1, 0, 0, 1, 0, ... | mammal] {boar},
 ...
]

In [8]:
X, mapping = OneHot.encode(data, include_class=True)

In [9]:
X

array([[False,  True,  True, ..., False,  True, False],
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ..., False, False, False],
       ..., 
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ...,  True, False, False],
       [ True, False, False, ..., False, False, False]], dtype=bool)

In [10]:
sorted(mapping.items())

[(0, (0, 0)),
 (1, (0, 1)),
 (2, (1, 0)),
 (3, (1, 1)),
 (4, (2, 0)),
 (5, (2, 1)),
 (6, (3, 0)),
 (7, (3, 1)),
 (8, (4, 0)),
 (9, (4, 1)),
 (10, (5, 0)),
 (11, (5, 1)),
 (12, (6, 0)),
 (13, (6, 1)),
 (14, (7, 0)),
 (15, (7, 1)),
 (16, (8, 0)),
 (17, (8, 1)),
 (18, (9, 0)),
 (19, (9, 1)),
 (20, (10, 0)),
 (21, (10, 1)),
 (22, (11, 0)),
 (23, (11, 1)),
 (24, (12, 0)),
 (25, (12, 1)),
 (26, (12, 2)),
 (27, (12, 3)),
 (28, (12, 4)),
 (29, (12, 5)),
 (30, (13, 0)),
 (31, (13, 1)),
 (32, (14, 0)),
 (33, (14, 1)),
 (34, (15, 0)),
 (35, (15, 1)),
 (36, (16, 0)),
 (37, (16, 1)),
 (38, (16, 2)),
 (39, (16, 3)),
 (40, (16, 4)),
 (41, (16, 5)),
 (42, (16, 6))]

In [11]:
itemsets = dict(frequent_itemsets(X, .4))

In [12]:
len(itemsets)

520

In [13]:
class_items = {item for item, var, _ in OneHot.decode(mapping, data, mapping) if var is data.domain.class_var}

In [14]:
sorted(class_items)

[36, 37, 38, 39, 40, 41, 42]

In [15]:
data.domain.class_var.values

['amphibian', 'bird', 'fish', 'insect', 'invertebrate', 'mammal', 'reptile']

In [16]:
rules = [(P, Q, supp, conf) for P, Q, supp, conf in association_rules(itemsets, .8) if len(Q) == 1 and Q & class_items]

In [17]:
len(rules)

18

In [18]:
rules

[(frozenset({2, 7, 17, 19, 20}), frozenset({41}), 41, 1.0),
 (frozenset({2, 7, 17, 19}), frozenset({41}), 41, 1.0),
 (frozenset({7, 17, 19, 20}), frozenset({41}), 41, 1.0),
 (frozenset({2, 7, 19, 20}), frozenset({41}), 41, 1.0),
 (frozenset({2, 17, 19, 20}), frozenset({41}), 41, 0.8723404255319149),
 (frozenset({2, 7, 17, 20}), frozenset({41}), 41, 1.0),
 (frozenset({2, 7, 20}), frozenset({41}), 41, 1.0),
 (frozenset({7, 17, 19}), frozenset({41}), 41, 1.0),
 (frozenset({2, 7, 17}), frozenset({41}), 41, 1.0),
 (frozenset({7, 17, 20}), frozenset({41}), 41, 1.0),
 (frozenset({7, 19, 20}), frozenset({41}), 41, 1.0),
 (frozenset({2, 17, 19}), frozenset({41}), 41, 0.8367346938775511),
 (frozenset({2, 7, 19}), frozenset({41}), 41, 1.0),
 (frozenset({7, 19}), frozenset({41}), 41, 1.0),
 (frozenset({7, 17}), frozenset({41}), 41, 1.0),
 (frozenset({2, 7}), frozenset({41}), 41, 1.0),
 (frozenset({7, 20}), frozenset({41}), 41, 1.0),
 (frozenset({7}), frozenset({41}), 41, 1.0)]

In [19]:
names = {item: '{}={}'.format(var.name, val) for item, var, val in OneHot.decode(mapping, data, mapping)}
for ante, cons, supp, conf in rules[:5]:
    print(', '.join(names[i] for i in ante), '-->', names[next(iter(cons))], '(supp: {}, conf: {})'.format(supp, conf))

backbone=1, feathers=0, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, feathers=0, breathes=1, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
feathers=0, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, feathers=0, breathes=1, venomous=0 --> type=mammal (supp: 41, conf: 0.8723404255319149)
